# Homework_2022_03_07

<font color=ff0000> 截止日期 3.12（周六） 23:59</font>

### 第一部分：命令模式（Command Pattern）

本周的课程我们学习了工厂模式，现在我们来看另一种面向对象的编程模式的应用，即**命令模式/Command Pattern**。命令模式可以将一个“请求”封装为一个对象，从而让我们**用不同的请求对客户进行参数化**，对请求进行排队，记录请求日志，或是取消已经请求的操作（参考：https://en.wikipedia.org/wiki/Command_pattern or https://blog.csdn.net/zzl_python/article/details/82732266 ）。

本次作业我们将用这种思路尝试完成一个简化后的股票交易所。Alice，Bob，Carol是三名股民，他们初始资金均为10000，交易所会记录他们提交的订单。为了简化模型，我们为股民免去了各种税费，且假设一共只有一支股票，每次的交易量均为1手（100股）。

在第一部分，请完成交易所框架的搭建，实现模拟交易的过程。
- 请完成框架，以BuyStockOrder类和SellStockOrder类的对象分别实现买卖股票的订单，并实例化三位股民；(1')
- 请重写Agent类的str方法，将交易所记录的历史交易保存为trade.log并打印；(1')
- 请重写StockTrade类的str方法，打印某一时刻各人的持股情况（初始持股数足够大，因此这里的持股情况可以小于0，且不用考虑t+1的限制）；(1')
- 请在StockTrade类中实现show_money方法，记录某一时刻各人的资金情况。(1')

有关交易的部分代码已经写好，请专注于体会命令模式的结构。

In [1]:
from abc import abstractmethod
class Order: # 抽象类，所有委托单都基于此类
    @abstractmethod
    def execute(): # 此处不需要修改，仅代表Order类的下单操作
        pass
    
class BuyStockOrder(Order): # 买单类
    def __init__(self, stock, name, price):
        self.order_type = 'Buy'
        self.stock = stock
        # TODO: 请完成对买单的初始化
        self.name = name
        self.price = price
        
    def execute(self):
        # TODO: 请调用StockTrade中的方法实现买单的执行
        self.stock.buy(self.name, self.price)
        
class SellStockOrder(Order): # 卖单类
    def __init__(self, stock, name, price):
        self.order_type = 'Sell'
        self.stock = stock
        # TODO: 请完成对卖单的初始化
        self.name = name
        self.price = price
        
    def execute(self):
        # TODO: 请调用StockTrade中的方法实现卖单的执行
        self.stock.sell(self.name, self.price)

class StockTrade: # 交易所，接受提交的买卖申请，并完成交易
    _instance = None # 启用单例模式，避免多个交易所同时存在
    def __new__(cls):
        if cls._instance is None:
            cls._instance = object.__new__(cls)
        return cls._instance
            
    def __init__(self):
        self.market = {} # 存储每个股民的持股情况
        self.money = {} # 存储每个股民的资金状况
        
    def buy(self, name, price): # 买入操作
        print(f'{name} bought the stock at {price}')
        if name in self.market.keys():
            self.market[name] += 100
            self.money[name] -= 100 * price
        else:
            self.market[name] = 100
            self.money[name] = 10000 - 100 * price
    def sell(self, name, price): # 卖出操作
        print(f'{name} sold the stock at {price}')
        if name in self.market.keys():
            self.market[name] -= 100
            self.money[name] += 100 * price
        else:
            self.market[name] = -100
            self.money[name] = 10000 + 100 * price
    
    def __str__(self):
        # TODO: 打印每个人的持股情况
        r = ''
        for i in self.market:
            r = r+i+': '+str(self.market[i])+'\n'
        return r.strip()
    
    def show_money(self):
        # TODO: 打印每个人的现金数
        r = ''
        for i in self.money:
            r = r+i+': '+str(self.money[i])+'\n'
        return r.strip()

class Agent: # 证券经纪人，记录买卖申请并向交易所提交
    def __init__(self):
        self.__OrderQueue = [] # 用于存储订单的队列
    
    def placeOrder(self, order):
        # TODO: 接受订单后的响应，包括1.存储订单入队列；2.执行订单（调用订单的execute方法）
        self.__OrderQueue.append(order)
        order.execute()
        
    def __str__(self):
        # TODO: 将所有订单的历史记录保存为trade.log并打印
        file = open('trade.log','w')
        r = ''
        for i in self.__OrderQueue:
            re = i.name+': '+i.order_type+' at '+str(i.price)+'\n'
            file.write(re)
            r = r+re
        return r.strip()

In [2]:
stock = StockTrade() # 初始化交易所（命令的执行者）
agent = Agent() # 初始化代理（命令的发出者）
# 让Alice，Bob和Carol试着交易一些股票，在这里不需要考虑买卖平衡的问题
agent.placeOrder(BuyStockOrder(stock, 'Alice', 10.01))
agent.placeOrder(SellStockOrder(stock, 'Bob', 10.15))
agent.placeOrder(SellStockOrder(stock, 'Carol', 10.43))
# 依次打印交易情况，持股情况和资金情况，所有订单的历史记录将同时保存为trade.log
print('\nHistory Order', agent, sep='\n')
print('\nCurrent Stock', stock, sep='\n')
print('\nCurrent Money', stock.show_money(), sep='\n')

Alice bought the stock at 10.01
Bob sold the stock at 10.15
Carol sold the stock at 10.43

History Order
Alice: Buy at 10.01
Bob: Sell at 10.15
Carol: Sell at 10.43

Current Stock
Alice: 100
Bob: -100
Carol: -100

Current Money
Alice: 8999.0
Bob: 11015.0
Carol: 11043.0


### 多进程通讯与文件读写
multiprocessing中的Pipe可以用于多进程之间的通讯，本题请用一个进程将上一步中保存的trade.log读取并发送至另一个进程，后者在交易记录开头写入当前时间并保存为trade_with_time.log。(2')

In [3]:
%%writefile pipe_hw.py # 由于多进程在Jupyter Notebook中可能无法运行，这里采用魔法方法writefile将此代码块保存为本地py文件

import time
from multiprocessing import Process, Pipe

def pipe_reader(pipe):
    with open('trade.log') as f:
        content = f.read()
        # TODO: 补全利用pipe发送信息部分
        pipe.send(content)
        pipe.close()

def pipe_writer(pipe):
    # TODO: 从pipe的另一端接收信息，在文件开头添加时间，并写入trade_with_time.log
    f = open('trade_with_time.log','w')
    f.write(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())+'\n')
    f.write(pipe.recv())
    
if __name__ == '__main__':
    # TODO: 创建Pipe，创建并执行两个子进程
    con_reader,con_writer = Pipe()
    pro1 = Process(target=pipe_reader,args=(con_reader,))
    pro2 = Process(target=pipe_writer,args=(con_writer,))
    pro1.start()
    pro1.join()
    pro2.start()
    pro2.join()


UsageError: unrecognized arguments: # 由于多进程在Jupyter Notebook中可能无法运行，这里采用魔法方法writefile将此代码块保存为本地py文件


In [ ]:
!python pipe_hw.py # 叹号代表执行Shell命令，在这里用于执行py文件实现多进程通讯

### 第二部分：完善交易系统

股票不会凭空产生也不会凭空消失，只是从一个人卖到另一个人手里。在第一部分，Alice，Bob，Carol三人在和整个市场做交易；这一部分，我们聚焦在这三人身上，假设交易只在这三人之间展开。

股票的交易遵循一定的基础规则，这里我们介绍连续竞价阶段**“价格优先”**和**“时间优先”**两个原则。

- 价格优先：**较高**价格的**买进**申请优先，**较低**价格的**卖出**申请优先。
- 时间优先：价格相同时，**先提交**的申请优先。

在本题所构建的交易所中，买单和卖单会分别存储并排序，每次提交订单后交易所将自动做出判断：当最高价买入单的价格等于最低价卖出单时，一笔交易将会按此价格发生；而如果最高买入价高于最低卖出价，交易也会发生，只不过交易的价格会和上一次的成交价有关，即所谓“撮合交易”。**假设买入价为X，卖出价为Y，上次成交价为Z，那么当且仅当X>=Y时本次交易会发生，且交易价格W = X(Y<=X<=Z) or Y(Z<=Y<=X) or Z(Y<=Z<=X)。** 若最高价买入单低于最低价卖出单，则不会有交易发生。

举例而言，如果股票市价（上次成交价）10.00元(Z)：Alice以9.98元(Y)售出，Bob以10.01元买入，Carol以10.02元(X)买入，最终的结果是Carol以10.00元(Y<Z<X, W=Z)买走了Alice的股票；Alice以10.01元(Y)卖出，Bob以10.03元(X)买入，Carol以10.02元买入，最终Bob以10.01元(Z<Y<X, W=Y)买走了Alice的股票；Alice以9.97元(Y)卖出，Bob以9.98元买入，Carol以9.99元(X)买入，最终Carol以9.99元(Y<X<Z, W=X)买走了Alice的股票。

在这一部分，Alice，Bob，Carol三人会不断下单，请重构Agent类，让交易所变得更真实，而这种提交之后对请求排队或修改的功能也正是命令模式的特色。
- 请完成撮合交易功能，在新的订单提交后检查是否有可以完成的交易，若有则进行交易（记得修改成交价）；(1')
- 在Agent类中实现history_price方法，记录每一次交易的成交价，同时在StockTrade类中实现show_property方法，将资金+股价\*股份数作为总资产记录；(1')
- 在之前的代码中我们采用了队列来实现订单列表，但队列在这里并不是效率最高的数据结构（思考：为什么?），请用**上一次作业实现的最小堆**（其他数据结构亦可）另外建立一套体系维护订单列表，要求能够提高效率，且满足价格优先原则(*时间优先原则本次作业不要求实现*)；请特别注意，用最小堆存储买单申请和存储卖单申请在细节上会有所区别；(1')

In [ ]:
from copy import deepcopy
class MinHeap: # 最小堆的代码已给出，以订单的价格进行排序；请注意这里存储的是订单而不仅是订单的价格
    # tips: 存储买单时可以将订单的价格取相反数，用最小堆同时实现最大堆的功能
    def __init__(self):
        self.heap_list = [BuyStockOrder(stock, '', 0)]
        self.current_size = 0

    def insert(self, k):
        self.heap_list.append(k)
        self.current_size+=1
        size=self.current_size
        self.sift_up(size)

    def sift_up(self, i):
        while i // 2 > 0:
            if self.heap_list[i].price < self.heap_list[i // 2].price:
                self.heap_list[i], self.heap_list[i // 2] = self.heap_list[i // 2], self.heap_list[i]
            i = i // 2

    def sift_down(self, i):
        while (i * 2) <= self.current_size:
            mc = self.min_child(i)
            if self.heap_list[i].price > self.heap_list[mc].price:
                self.heap_list[i], self.heap_list[mc] = self.heap_list[mc], self.heap_list[i]
            i = mc

    def min_child(self, i):
        if (i * 2) + 1 > self.current_size:
            return i * 2
        else:
            if self.heap_list[i * 2].price < self.heap_list[(i * 2) + 1].price:
                return i * 2
            else:
                return (i * 2) + 1

    def delete_min(self):
        if len(self.heap_list) == 1:
            return 'Empty heap'

        root = self.heap_list[1]
        self.heap_list[1] = self.heap_list[self.current_size]
        *self.heap_list, _ = self.heap_list
        self.current_size -= 1

        self.sift_down(1)
        return root
    
    def sort(self):
        ls=[]
        while(len(self.heap_list)>1):
            if self.heap_list[1].price > 0:
                ls.append(f'{self.heap_list[1].name}: {self.heap_list[1].price}')
            else: # 如果存储的订单价格为负（相反数），输出列表时记得再次取相反数
                ls.append(f'{self.heap_list[1].name}: {-self.heap_list[1].price}') 
            self.delete_min()
        return ls
    
class StockTrade:
    _instance = None # 启用单例模式，避免多个交易所同时存在
    def __new__(cls):
        if cls._instance is None:
            cls._instance = object.__new__(cls)
        return cls._instance
    
    def __init__(self):
        self.market = {} # 存储每个股民的持股情况
        self.money = {} # 存储每个股民的资金状况
        self.price = 10 # 初始价格
        
    def buy(self, name, price):
        self.price = price
        print(f'{name} bought the stock at {price}')
        if name in self.market.keys():
            self.market[name] += 100
            self.money[name] -= 100 * price
        else:
            self.market[name] = 100
            self.money[name] = 10000 - 100 * price
    def sell(self, name, price):
        self.price = price
        print(f'{name} sold the stock at {price}')
        if name in self.market.keys():
            self.market[name] -= 100
            self.money[name] += 100 * price
        else:
            self.market[name] = -100
            self.money[name] = 10000 + 100 * price
    
    def __str__(self):
        # TODO: 打印每个人的持股情况
        r = ''
        for i in self.market:
            r = r + i+ ': ' + str(self.market[i]) + '\n'
        return r.strip()
    
    def show_money(self):
        # TODO: 打印每个人的现金数
        r = ''
        for i in self.money:
            r = r + i + ': ' + str(self.money[i]) + '\n'
        return r.strip()
    
    def show_property(self):
        # TODO: 打印每个人的总资产（现金+持股*市价）
        r = ''
        for i in self.money:
            r = r + i + ': ' + str(self.money[i] + self.market[i]*self.price) + '\n'
        return r.strip()
    
class Agent:
    def __init__(self):
        self.__OrderQueue = [] # 用于按提交顺序存储订单的队列
        self.__BuyQueue = MinHeap() # TODO; 请选用合适的数据结构存储买单，保持价格最高的买单在最前
        self.__SellQueue = MinHeap() # TODO: 类似地，保持价格最低的卖单在最前
        self.price = [10] # 存储历史价格
        
    def placeOrder(self, order):
        order.price = round(order.price, 2)
        self.__OrderQueue.append(deepcopy(order))
        if order.order_type == 'Buy':
            # TODO: 完成对买单的存储，保持最高价买入单在前（tips: 若用最小堆可以将价格取相反数存储）
            order.price = -order.price
            self.__BuyQueue.insert(order)
        else:
            # TODO: 完成对卖单的存储，保持最低价卖出单在前
            self.__SellQueue.insert(order)
        if len(self.__BuyQueue.heap_list) > 1 and len(self.__SellQueue.heap_list) > 1:
            # TODO: 判断能否交易，计算成交价，执行买卖操作，更新买卖队列（维护最小堆），更新历史价格
            Ob=self.__BuyQueue.delete_min()
            Os=self.__SellQueue.delete_min()
            Ob.price=-Ob.price

            # can do
            if Ob.price >= Os.price: 
                now_P = last_P
                last_P = self.price[-1]
                
                if last_P >= Os.price:
                    now_P = Ob.price
                elif last_P <= Ob.price:
                    now_P = Os.price
                    
                Os.price=now_P
                Ob.price=now_P
                
                Ob.execute()
                Os.execute()
                self.price.append(now_P)
            else:
                Ob.price=-Ob.price
                self.__BuyQueue.insert(Ob)
                self.__SellQueue.insert(Os)    
    def show_buy(self):
        return '\n'.join(deepcopy(self.__BuyQueue).sort())
    
    def show_sell(self):
        return '\n'.join(deepcopy(self.__SellQueue).sort())
        
    def history_price(self):
        # TODO: 依次打印历史价格
        return self.price
    
    def __str__(self):
        # TODO: 将所有订单的历史记录保存为trade.log并打印
        f = open('trade.log','w')
        a = ''
        for i in self.__OrderQueue:
            r = i.name+': Tried to '+i.order_type+' at '+str(i.price)+'\n'
            f.write(r)
            a = a+r
        return a.strip()

In [ ]:
stock = StockTrade()
agent = Agent()
# 让Alice，Bob和Carol试着交易一些股票，不一定局限于这里给出的例子
agent.placeOrder(BuyStockOrder(stock, 'Alice', 10.01))
agent.placeOrder(SellStockOrder(stock, 'Bob', 9.99))
agent.placeOrder(SellStockOrder(stock, 'Carol', 10.02))
agent.placeOrder(BuyStockOrder(stock, 'Alice', 10.03))
agent.placeOrder(BuyStockOrder(stock, 'Bob', 10.01))
agent.placeOrder(SellStockOrder(stock, 'Carol', 10.02))
# 依次打印交易历史，持股情况，资产情况，剩余买单，剩余卖单，历史价格
print('\nHistory Order', agent, sep='\n')
print('\nCurrent Stock', stock, sep='\n')
print('\nCurrent Property', stock.show_property(), sep='\n')
print('\nRemaining Buy Order', agent.show_buy(), sep='\n')
print('\nRemaining Sell Order', agent.show_sell(), sep='\n')
print('\nHistory Price', agent.history_price(), sep='\n')

### 多线程(1')
**请先完成前面交易所的框架与撮合交易系统。**

现实中的交易并不会是各人轮流按顺序进行，为了更接近真实的交易情况，请为Alice，Bob，Carol三人设计各自独立的交易体系，并让三人同时高频参与交易。

交易所stock与代理商agent对三人是公共的资源，在前面的基础上，三人的交易申请已经封装为execute_order直接调用即可。**每人可以进场一千次，进场时可以用任意价格挂买单/卖单/什么都不做。**
- 请用多线程（推荐，但多进程也可）的方法实现三人的高频交易，每人对应一个线程/进程(1')；

### 附加题(+1'~2')
**请先完成多线程部分任务。**

在前一部分的交易中，Alice的交易策略是随机买入卖出，Bob会利用股价变化低吸高抛，在股票下跌时买入，上涨时卖出；现在请你替代Carol的位置，参与这场游戏。

作为人类玩家，你可以用agent.history_price()调取历史股价，也可以用agent.show_buy() & agent.show_sell()查看目前未交易的买单/卖单，你可以充分利用你拥有的信息，以此选择买入/卖出的时机与价格。
- 请编写你的交易策略，在一千轮之后用总资产战胜 *随机交易的Alice* 和 *贪心算法的Bob* (+1' for 比较稳定地胜出/+2' for 赢麻了)。

提示：请记得检查交易结束后三人的总持股数之和是否为零，总资产之和是否为30000，否则可能说明程序有误。

In [ ]:
import threading
import time
from random import randint, random

lock = threading.Lock() # 多个线程同时访问公共变量需要加锁（思考：如果不加会怎么样？）

interval = .001 # 如果跑不动/交易很混乱，可以试着在每个人的交易之后加一个短暂的interval

def execute_order(name, s, a):
    stock = s
    agent = a
    if name == 'Alice':
        for i in range(1000): # Alice随机交易，买入卖出概率各为一半，价格随机围绕市价浮动
            rand_type, rand_price = random(), randint(-2, 5)
            lock.acquire()
            if rand_type > 0.5:
                agent.placeOrder(BuyStockOrder(stock, name, agent.history_price()[-1] + 0.01 * rand_price))
            else:
                agent.placeOrder(SellStockOrder(stock, name, agent.history_price()[-1] - 0.01 * rand_price))
            lock.release()
            time.sleep(interval)
    elif name == 'Bob': # Bob采用贪心算法，股票价格下降时购买，上升时卖出
        for i in range(1000):
            lock.acquire()
            if len(agent.history_price()) > 1:
                if agent.history_price()[-1] > agent.history_price()[-2]:
                    agent.placeOrder(SellStockOrder(stock, name, agent.history_price()[-1] + 0.01))
                elif agent.history_price()[-1] < agent.history_price()[-2]:
                    agent.placeOrder(BuyStockOrder(stock, name, agent.history_price()[-1] - 0.01))
            lock.release()
            time.sleep(interval)
    elif name == 'Carol': 
        # 请先保持这里不变，实现多线程提交订单后，可以观赏一下Bob如何用简单的策略战胜Alice和Carol脱颖而出
        # 附加题TODO：在完成多线程后可以直接修改这里的内容，选择Carol的买卖条件与定价策略，战胜Alice和Bob（如果不做附加题则不用修改）
        for i in range(1000):
            rand_type, rand_price = random(), randint(-2, 5)
            lock.acquire()
            if rand_type > 0.5: # 附加题TODO：买入条件
                agent.placeOrder(BuyStockOrder(stock, name, 10000))
            else: # 附加题TODO：卖出条件
                agent.placeOrder(BuyStockOrder(stock, name, 10000))
            lock.release()
            time.sleep(interval)
        
stock = StockTrade()
agent = Agent()
threads=[]
for name in ['Alice', 'Bob', 'Carol']:
    t = threading.Thread(target=execute_order,args=(name,stock,agent,))
    threads.append(t)
for t in threads:
    t.start()
for t in threads:
    t.join()
print('\nHistory Price', agent.history_price(), sep='\n')
print('\nCurrent Stock', stock, sep='\n') # 最后总持股数之和应为0
print('\nCurrent Property', stock.show_property(), sep='\n') # 最后总资产之和应为30000